In [1]:
#import multiple files with names as Kickstarter*.csv into pandas dataframe (data used 15 Aug 2019)

from glob import glob
import pandas as pd

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

#just replace the path parts
#glob import all files starting with Kickstarter in the folder (the folder i used is the 15 Aug 2019)

filenames = glob(r"Kickstarter_2019-08-15T03_20_03_022Z/Kickstarter*.csv")

df = [pd.read_csv(f, encoding='utf-8') for f in filenames]

data = pd.concat(df, axis = 0, ignore_index = True).reset_index()

In [2]:
from pandas.io.json import json_normalize
import json as json
import numpy as np

#data['category'].fillna(json.dumps('{}'), inplace = True)
data['location'].fillna('{}', inplace = True)
#data['creator'].fillna(json.dumps('{}'), inplace = True)

x1 = json_normalize(data['category'].apply(json.loads)).add_prefix('category_')
x3 = json_normalize(data['location'].apply(json.loads)).add_prefix('location_')
x5 = json_normalize(data['creator'].apply(lambda x: json.loads(x.replace('\n', '').replace('\r', '').replace('\\', '').replace('"', r'\"').replace(r'{\"', '{"').replace(r'\"}', '"}').replace(r':\"', ':"').replace(r'\":', '":').replace(r',\"', ',"').replace(r'\",', '",').replace(r'", ', r'\",')))).add_prefix('creator_')

data = pd.concat([data, x1, x3, x5], axis = 1)

In [3]:
from datetime import datetime
data['launched_at_utc'] = data['launched_at'].apply(datetime.utcfromtimestamp)
data['deadline_utc'] = data['deadline'].apply(datetime.utcfromtimestamp)
data['created_at_utc'] = data['created_at'].apply(datetime.utcfromtimestamp)
data['goal'] = round(data['goal'] * data['static_usd_rate'], 2)

In [4]:
#pick the relevant data and drop all irrelevant data 

df = data[[ 
'goal', 'state', 'location_id',  'category_id', 'deadline_utc', 'launched_at_utc']]

#feature creation 
#feature 1: no. of projects for each creator 
# df1 = df.groupby('creator_id')['id'].count()
# df = pd.merge(df, pd.DataFrame(df1), how = 'left', on = 'creator_id')

#feature 2: duration of project 
df['duration'] = (df['deadline_utc'] - df['launched_at_utc']).dt.days

#feature 4: month of launch
df['launch_month'] = df['launched_at_utc'].dt.month

#change state to numerical 
df['state'] = df['state'].map({'successful': 1, 'failed': 0 })

# #change staff_pick to numerical 
# df['staff_pick'] = df['staff_pick'].map({True: 1, False: 0 })

#drop columns
df = df.drop(columns = ['launched_at_utc', 'deadline_utc'])
df = df.dropna() #drop null rows
columns = df.columns.tolist()

columns = columns[-4:] + columns[:-4]
df = df[columns]

/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [5]:
print(df.columns.tolist())
print(df.iloc[0].values)
X = np.array(df.iloc[:,:-1].values)
y = np.array(df.iloc[:,-1].values)
print(X.shape)
print(y.shape)

['location_id', 'category_id', 'duration', 'launch_month', 'goal', 'state']
[2.483145e+06 2.700000e+01 2.900000e+01 2.000000e+00 5.000000e+02
 0.000000e+00]
(192137, 5)
(192137,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape)

(153709, 5) (38428, 5)


In [7]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
print('After feature scaling: ', X_train[10])

After feature scaling:  [-0.12502847 -0.99374277 -0.19165208  1.74355751 -0.02613001]


In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential()

input_dim = int(X_train.shape[1])
output_dim = int(X_train.shape[1] / 2)

print('Input dim: ', input_dim)
print('Output dim: ', output_dim)

#input layer
classifier.add(Dense(units=output_dim, activation='relu', input_dim=input_dim))
# classifier.add(Dropout(p=0.1))

# hidden layer
classifier.add(Dense(units=output_dim, activation='relu'))
# classifier.add(Dropout(p=0.1))

# output layer
classifier.add(Dense(output_dim=1, activation='sigmoid'))

# compile ann
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fit ANN
classifier.fit(X_train, y_train, batch_size=50, epochs=50)

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Input dim:  5
Output dim:  2


/home/muruges/.conda/envs/CS3244_Proj/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`


Epoch 1/50
153709/153709 [==============================] - 1s 9us/step - loss: 0.6304 - acc: 0.6524
Epoch 2/50
153709/153709 [==============================] - 1s 9us/step - loss: 0.6018 - acc: 0.6810
Epoch 3/50
153709/153709 [==============================] - 1s 9us/step - loss: 0.5966 - acc: 0.6848A: 0s - loss: 0.5973 - 
Epoch 4/50
153709/153709 [==============================] - 1s 9us/step - loss: 0.5929 - acc: 0.6881
Epoch 5/50
153709/153709 [==============================] - 1s 8us/step - loss: 0.5915 - acc: 0.6891
Epoch 6/50
153709/153709 [==============================] - 1s 8us/step - loss: 0.5912 - acc: 0.6907
Epoch 7/50
153709/153709 [==============================] - 1s 9us/step - loss: 0.5911 - acc: 0.6904
Epoch 8/50
153709/153709 [==============================] - 1s 9us/step - loss: 0.5911 - acc: 0.6903
Epoch 9/50
153709/153709 [==============================] - 1s 8us/step - loss: 0.5911 - acc: 0.6898
Epoch 10/50
153709/153709 [==============================] - 1s 8us/

In [9]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [False],
       [ True],
       [ True]])

In [10]:
from sklearn.metrics import accuracy_score
metrics = accuracy_score(y_test, y_pred)
metrics

0.6911106484854793

In [ ]:
3